-------------------------------------------------------------------------------------------------------------------------------
# TE-276: LAB 2 - Versão do Aluno
-------------------------------------------------------------------------------------------------------------------------------

============================================================
============================================================
# Install py-trees

In [ ]:
!pip install py-trees==2.2.3

============================================================
============================================================
# Install celluloid

In [ ]:
!pip install celluloid==0.2.0

============================================================
============================================================

# Two-dimensional motion of a drone:

- modulus of the velocity (v)

- orientation angle (alfa)

- vehicle's position (p = (x,y))


x(t) = x(t-1) + v(t-1) * seno(alfa(t-1))

y(t) = y(t-1) + v(t-1) * cosseno(alfa(t-1))
 

<div>
<img src="drone.jpg" width="400"/>
</div>

In [ ]:
import numpy
import math
import py_trees
from matplotlib import pyplot
import time
from celluloid import Camera


#----------------------------------------------------------------------------------------------------------------------
# Environment parameters.
#----------------------------------------------------------------------------------------------------------------------
environment = {
    # Limiting coordinates of the environment
    # x belongs to the interval [0.0 , 100.0]
    # y belongs to the interval [0.0 , 100.0]
    "limiting coordinates":{
        "min_x":0.0,
        "max_x":100.0,
        "min_y":0.0,
        "max_y":100.0
    },
    # Radiation
    "radiation":{
        "radiation sources":[
            ["source 1",40.0,40.0,10.0] #[source name, x, y, radiation range]
        ],
        "activated_radiation_source":False
    },
    # Broadcast channel
    "broadcast":[]
}
#----------------------------------------------------------------------------------------------------------------------


class Energy_Sensor:
    
    def __init__(self, energy_level):
        self.energy_level = energy_level
    
    def Read_Energy_Level(self):
        return self.energy_level
    
    def Updade_Energy_Level(self,energy_consumption):
        self.energy_level = self.energy_level - energy_consumption
        
    def Set_Energy_Level(self, new_energy_level):
        self.energy_level = new_energy_level

        
class Damage_Sensor:
    
    def __init__(self, damage_level):
        self.damage_level = damage_level
    
    def Read_Damage_Level(self):
        return self.damage_level
    
    def Updade_Damage_Level(self,damage_factor,damage_percentage):
        self.damage_level = self.damage_level + damage_factor * damage_percentage
        
    def Set_Damage_Level(self, new_damage_level):
        self.damage_level = new_damage_level
        

class Radiation_Sensor:
    
    def Read_Radiation_Level(self, x, y):        
              
        radiation_level = 0.0
        
        if (environment["radiation"]["activated_radiation_source"] == True):
                    
            for index_radiation_source in range(len(environment["radiation"]["radiation sources"])):
                         
                distance_agent_radiation_source = math.sqrt((x-environment["radiation"]["radiation sources"][index_radiation_source][1])*
                                                            (x-environment["radiation"]["radiation sources"][index_radiation_source][1]) +
                                                            (y-environment["radiation"]["radiation sources"][index_radiation_source][2])*
                                                            (y-environment["radiation"]["radiation sources"][index_radiation_source][2]))
        
                if (distance_agent_radiation_source < environment["radiation"]["radiation sources"][index_radiation_source][3]):
                    # 0.0 < radiation_level_per_source <= 1.0
                    radiation_level_per_source = 1.0 - distance_agent_radiation_source / environment["radiation"]["radiation sources"][index_radiation_source][3]        
                    radiation_level = radiation_level + radiation_level_per_source
        
        return radiation_level


class Drone:
    
    #----------------------------------------------------------------------------------------------------------------------
    # Internal parameters
    #----------------------------------------------------------------------------------------------------------------------
    energy_consumption = 0.05 # % per iteration.
    limit_energy_refueling = 40.0 # %.
    refueling_rate = 2.0 # % per iteration.
    maximum_control_signal_orientation_angle = 1.0 # degrees.
    maximum_control_signal_velocity = 0.1 # units per iteration.
    control_signal_orientation_angle = 0.0 # variation of the orientation angle, in degrees.
    control_signal_velocity = 0.0 # variation of the velocity, in units per iteration.
    factor_radiation_damage = 0.1 # damage per radiation level equal to 100%.
    limit_damage = 70.0 # %.
    repair_rate = 1.0 # damage level deacreased per iteration.
    reference_velocity = 0.1 # units per iteration
    
    # Construtor
    def __init__(self, ID):
        self.ID = ID
        
        self.action = {
            'required_orientation_angle':0.0, # degrees
            'required_velocity': 0.1 # units per iteration
        }
    
        #----------------------------------------------------------------------------------------------------------------------
        # Sensors
        #----------------------------------------------------------------------------------------------------------------------
        self.energy_sensor = Energy_Sensor(100.0)
    
        self.damage_sensor = Damage_Sensor(0.0)
    
        self.radiation_sensor = Radiation_Sensor()
        #----------------------------------------------------------------------------------------------------------------------
    
    
        #----------------------------------------------------------------------------------------------------------------------
        # State of the drone
        #----------------------------------------------------------------------------------------------------------------------
        self.state = {
            'x':0.0,
            'y':0.0,
            'velocity':0.1,
            'orientation_angle':0.0,
            'energy_level':100.0,
            'damage_level': 0.0
        }
        #----------------------------------------------------------------------------------------------------------------------
        
        #----------------------------------------------------------------------------------------------------------------------
        # Blackboard
        #----------------------------------------------------------------------------------------------------------------------
        self.blackboard = {
            'number_ticks':0,
            'index_current_surveillance_position': 1,
            'x_surveillance_position_1': 70.0,
            'y_surveillance_position_1': 80.0, 
            'x_surveillance_position_2': 70.0,
            'y_surveillance_position_2': 20.0,
            'x_refueling_position': 10.0,
            'y_refueling_position': 50.0, 
            'x_current_destination_position': 70.0,
            'y_current_destination_position': 80.0
        }
        #----------------------------------------------------------------------------------------------------------------------
        
        #----------------------------------------------------------------------------------------------------------------------
        # Root of the behavior tree
        #----------------------------------------------------------------------------------------------------------------------
        self.behavior_tree_root = None
        #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def S(self,x1, y1, x2, y2, x3, y3):
    
        s = (y2 - y1) * (x3 - x2) - (y3 - y2) * (x2 - x1)
    
        return s
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def Get_Sensors_Data(self):
        
        #---------------------------------------------------
        # Get data from radiation sensor.
        radiation_level = self.radiation_sensor.Read_Radiation_Level(self.state['x'],self.state['y'])
        #---------------------------------------------------
        
        #---------------------------------------------------
        # Update and get data from damage sensor. Observation: the drone can suffer damage due to different causes.
        self.damage_sensor.Updade_Damage_Level(self.factor_radiation_damage,radiation_level)
        self.state['damage_level'] =  self.damage_sensor.Read_Damage_Level()
        #---------------------------------------------------
        
        #---------------------------------------------------
        # Update and get data from energy sensor.
        self.energy_sensor.Updade_Energy_Level(self.energy_consumption)
        self.state['energy_level'] =  self.energy_sensor.Read_Energy_Level()
        #---------------------------------------------------
        
        
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def Update_Motion_Model(self):
                
        # Update the position.
        self.state['x'] = self.state['x'] + self.state['velocity'] * math.cos(self.state['orientation_angle'] * math.pi / 180.0)
        
        self.state['y'] = self.state['y'] + self.state['velocity'] * math.sin(self.state['orientation_angle'] * math.pi / 180.0)
        
        # Update the orientation angle.
        self.state['orientation_angle'] = self.state['orientation_angle'] + self.control_signal_orientation_angle
        
        # Update the velocity.
        if ((self.state['velocity'] == self.maximum_control_signal_velocity) and
            (self.control_signal_velocity == self.maximum_control_signal_velocity)):
            self.control_signal_velocity = 0.0
        self.state['velocity'] = self.state['velocity'] + self.control_signal_velocity
        
        if (self.state['orientation_angle'] < 0.0):
            self.state['orientation_angle'] = 360.0 + self.state['orientation_angle']
        else:
            if (self.state['orientation_angle'] > 360.0):
                self.state['orientation_angle'] = self.state['orientation_angle'] - 360.0
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def Update_State(self, t):
    
        if (self.state['energy_level'] > 0.0):
            
            self.Get_Sensors_Data()
            
            self.Update_Motion_Model()
        
        else:
            
            print(">>>> PROBLEM: drone without energy <<<<")
    #----------------------------------------------------------------------------------------------------------------------
    
    #----------------------------------------------------------------------------------------------------------------------
    # Systems
    #----------------------------------------------------------------------------------------------------------------------
    def Control_System(self):
        
        # Control signal to stop the drone.
        if (self.action['required_velocity'] == 0.0):
            self.control_signal_velocity = 0.0
            if (self.state['velocity'] > 0.0):
                self.control_signal_velocity = -1.0*self.maximum_control_signal_velocity
        
        # Control signal to specify the drone velocity equal to a reference velocity.
        if (self.action['required_velocity'] == self.reference_velocity):
            self.control_signal_velocity = 0.0
            if (self.state['velocity'] < self.reference_velocity):
                self.control_signal_velocity = 1.0*self.maximum_control_signal_velocity
            if (self.state['velocity'] > self.reference_velocity):
                self.control_signal_velocity = -1.0*self.maximum_control_signal_velocity
        
        # Control signal to specify the drone orientation angle.
        difference1 = abs(self.action['required_orientation_angle'] - self.state['orientation_angle']) 
        difference2 = abs(360.0 - self.action['required_orientation_angle'] + self.state['orientation_angle'])
        difference3 = abs(360.0 - self.state['orientation_angle'] + self.action['required_orientation_angle'])
        if (self.state['orientation_angle'] < self.action['required_orientation_angle']):
            if (difference1 < difference2):
                # Turn left
                self.control_signal_orientation_angle = self.maximum_control_signal_orientation_angle
            else: # (difference1 >= difference2)
                # Turn right
                self.control_signal_orientation_angle = -1.0*self.maximum_control_signal_orientation_angle
        else:
            if (self.state['orientation_angle'] > self.action['required_orientation_angle']):
                if (difference1 < difference3):
                    # Turn right
                    self.control_signal_orientation_angle = -1.0*self.maximum_control_signal_orientation_angle
                else: # (difference1 >= difference3):
                    # Turn left
                    self.control_signal_orientation_angle = self.maximum_control_signal_orientation_angle
            else:
                if (self.state['orientation_angle'] == self.action['required_orientation_angle']):
                    self.control_signal_orientation_angle = 0.0
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    # Behavior tree of the drone
    #----------------------------------------------------------------------------------------------------------------------
    
    # Definition of the classes of the behavior tree nodes.
    # Each class is the type of a condiction or action node.
    
    #++++++++
    class Condition_Destination_Reached(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Condition_Destination_Reached"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Condition_Destination_Reached, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Condition_Destination_Reached")
        
            ##########
            distance_limit = 0.5
            
            distance_agent_destination = math.sqrt((self.autonomous_agent.state['x']-self.autonomous_agent.blackboard['x_current_destination_position'])*
                                                  (self.autonomous_agent.state['x']-self.autonomous_agent.blackboard['x_current_destination_position']) +
                                                  (self.autonomous_agent.state['y']-self.autonomous_agent.blackboard['y_current_destination_position'])*
                                                  (self.autonomous_agent.state['y']-self.autonomous_agent.blackboard['y_current_destination_position']))
            
            if (distance_agent_destination <= distance_limit):
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Action_Set_Destination_Equal_To_Other_Surveillance_Position_S(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Set_Destination_Equal_To_Other_Surveillance_Position_S"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Set_Destination_Equal_To_Other_Surveillance_Position_S, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Set_Destination_Equal_To_Other_Surveillance_Position_S")
        
            ##########
            if (self.autonomous_agent.blackboard['index_current_surveillance_position'] == 1):
                self.autonomous_agent.blackboard['index_current_surveillance_position'] = 2
                self.autonomous_agent.blackboard['x_current_destination_position'] = self.autonomous_agent.blackboard['x_surveillance_position_2']
                self.autonomous_agent.blackboard['y_current_destination_position'] = self.autonomous_agent.blackboard['y_surveillance_position_2']
            else:
                self.autonomous_agent.blackboard['index_current_surveillance_position'] = 1
                self.autonomous_agent.blackboard['x_current_destination_position'] = self.autonomous_agent.blackboard['x_surveillance_position_1']
                self.autonomous_agent.blackboard['y_current_destination_position'] = self.autonomous_agent.blackboard['y_surveillance_position_1']
            
            new_status = (py_trees.common.Status.SUCCESS)
            ##########
            
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Action_Navigate_To_Destination_A(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Navigate_To_Destination_A"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Navigate_To_Destination_A, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Navigate_To_Destination_A")
        
            ##########
            # Velocity is specified equal to the reference velocity aiming to avoid velocity changing.
            self.autonomous_agent.action['required_velocity'] = self.autonomous_agent.reference_velocity
        
            # Calculate the auxiliary future position.
            x_future_position = self.autonomous_agent.state['x'] + math.cos(self.autonomous_agent.state['orientation_angle']*math.pi/180.0)
            y_future_position = self.autonomous_agent.state['y'] + math.sin(self.autonomous_agent.state['orientation_angle']*math.pi/180.0)
    
            # u1 is the unit vector based on the vector (agent's position, future position)
            x_u1 = x_future_position - self.autonomous_agent.state['x']
            y_u1 = y_future_position - self.autonomous_agent.state['y']
            
            # u2 is the unit vector based on the vector (agent's position, destination)
            x_u2 = self.autonomous_agent.blackboard['x_current_destination_position'] - self.autonomous_agent.state['x']
            y_u2 = self.autonomous_agent.blackboard['y_current_destination_position'] - self.autonomous_agent.state['y']
            
            # Angle alfa between the unit vectors u1 and u2.
            alfa = math.acos( (x_u1*x_u2 + y_u1*y_u2) / (math.sqrt(x_u1*x_u1 + y_u1*y_u1) * math.sqrt(x_u2*x_u2 + y_u2*y_u2)) )        
            
            alfa = alfa*180.0/math.pi
        
            # Side of the destination position relative to the navigation direction of the drone (heading).
            s = self.autonomous_agent.S(self.autonomous_agent.state['x'],
                                        self.autonomous_agent.state['y'],
                                        x_future_position,
                                        y_future_position,
                                        self.autonomous_agent.blackboard['x_current_destination_position'],
                                        self.autonomous_agent.blackboard['y_current_destination_position'])
    
            
            self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle']
            
            if (s < 0.0): # counterclockwise
                # Turn left
                self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle'] + alfa
            else:
                if (s > 0): # clockwise
                    # Turn right
                    self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle'] - alfa
                else:
                    if (s == 0.0):
                        self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle']
            
            if (self.autonomous_agent.action['required_orientation_angle'] < 0.0):
                self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.action['required_orientation_angle'] + 360.0
            if (self.autonomous_agent.action['required_orientation_angle'] > 360.0):
                self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.action['required_orientation_angle'] - 360.0
            
            new_status = (py_trees.common.Status.RUNNING)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Condition_Low_Energy_Level(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Condition_Low_Energy_Level"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Condition_Low_Energy_Level, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Condition_Low_Energy_Level")
        
            ##########
            if (self.autonomous_agent.state['energy_level'] <= self.autonomous_agent.limit_energy_refueling):
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Action_Set_Destination_Equal_To_Refueling_Position_S(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Set_Destination_Equal_To_Refueling_Position_S"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Set_Destination_Equal_To_Refueling_Position_S, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Set_Destination_Equal_To_Refueling_Position_S")
        
            ##########
            self.autonomous_agent.blackboard['x_current_destination_position'] = self.autonomous_agent.blackboard['x_refueling_position']
            self.autonomous_agent.blackboard['y_current_destination_position'] = self.autonomous_agent.blackboard['y_refueling_position']
            
            new_status = (py_trees.common.Status.SUCCESS)
            ##########
            
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Action_Set_Destination_Equal_To_Current_Surveillance_Position_S(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Set_Destination_Equal_To_Current_Surveillance_Position_S"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Set_Destination_Equal_To_Current_Surveillance_Position_S, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Set_Destination_Equal_To_Current_Surveillance_Position_S")
        
            ##########
            if (self.autonomous_agent.blackboard['index_current_surveillance_position'] == 1):
                self.autonomous_agent.blackboard['x_current_destination_position'] = self.autonomous_agent.blackboard['x_surveillance_position_1']
                self.autonomous_agent.blackboard['y_current_destination_position'] = self.autonomous_agent.blackboard['y_surveillance_position_1']
            else:
                self.autonomous_agent.blackboard['x_current_destination_position'] = self.autonomous_agent.blackboard['x_surveillance_position_2']
                self.autonomous_agent.blackboard['y_current_destination_position'] = self.autonomous_agent.blackboard['y_surveillance_position_2']
            
            new_status = (py_trees.common.Status.SUCCESS)
            ##########
            
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
            
    #++++++++
    class Action_Refuel_A(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Refuel_A"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Refuel_A, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Refuel_A")
        
            ##########
            # Required velocity is specified equal to zero to stop the drone motion.
            self.autonomous_agent.action['required_velocity'] = 0.0
            
            # Required orientation angle is specified equal to the current orientation angle.            
            self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle']
            
            new_status = (py_trees.common.Status.RUNNING)
            
            self.autonomous_agent.energy_sensor.Set_Energy_Level(self.autonomous_agent.energy_sensor.Read_Energy_Level() + self.autonomous_agent.refueling_rate)
            
            if (self.autonomous_agent.energy_sensor.Read_Energy_Level() > 100.0):
                self.autonomous_agent.energy_sensor.Set_Energy_Level(100.0)
            
            if (self.autonomous_agent.energy_sensor.Read_Energy_Level() == 100.0):
                # Control signal to restart the drone motion with the reference velocity.
                self.autonomous_agent.action['required_velocity'] = self.autonomous_agent.reference_velocity
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Condition_Radiation_Detected(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Condition_Radiation_Detected"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Condition_Radiation_Detected, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Condition_Radiation_Detected")
        
            ##########
            if (self.autonomous_agent.radiation_sensor.Read_Radiation_Level(self.autonomous_agent.state['x'], self.autonomous_agent.state['y']) > 0.0):
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Condition_High_Damage_Level(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Condition_High_Damage_Level"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Condition_High_Damage_Level, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Condition_High_Damage_Level")
        
            ##########
            if (self.autonomous_agent.state['damage_level'] >= self.autonomous_agent.limit_damage):
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    
    #++++++++
    class Action_Repair_Damage_A(py_trees.behaviour.Behaviour):
        
        def __init__(self, autonomous_agent, name: str = "Action_Repair_Damage_A"):
            self.autonomous_agent = autonomous_agent
            #Configure the name of the behaviour.
            super(self.autonomous_agent.Action_Repair_Damage_A, self).__init__(name)
            self.logger.debug("%s.__init__()" % (self.__class__.__name__))

        def setup(self, **kwargs: int) -> None:
            #No delayed initialisation required for this example.
            self.logger.debug("%s.setup()" % (self.__class__.__name__))

        def initialise(self) -> None:
            #Reset a counter variable.
            self.logger.debug("%s.initialise()" % (self.__class__.__name__))

        def update(self) -> py_trees.common.Status:
            new_status = (py_trees.common.Status.FAILURE)
        
            self.autonomous_agent.blackboard['number_ticks'] = self.autonomous_agent.blackboard['number_ticks'] + 1
    
            #print("tick on the node Action_Repair_Damage_A")
        
            ##########
            # Required velocity is specified equal to zero to stop the drone motion.
            self.autonomous_agent.action['required_velocity'] = 0.0
            
            # Required orientation angle is specified equal to the current orientation angle.            
            self.autonomous_agent.action['required_orientation_angle'] = self.autonomous_agent.state['orientation_angle']
            
            new_status = (py_trees.common.Status.RUNNING)
            
            self.autonomous_agent.damage_sensor.Set_Damage_Level(self.autonomous_agent.damage_sensor.Read_Damage_Level() - self.autonomous_agent.repair_rate)
            
            if (self.autonomous_agent.damage_sensor.Read_Damage_Level() < 0.0):
                self.autonomous_agent.damage_sensor.Set_Damage_Level(0.0) 
            
            if (self.autonomous_agent.damage_sensor.Read_Damage_Level() == 0.0):
                # Control signal to restart the drone motion with the reference velocity.
                self.autonomous_agent.action['required_velocity'] = self.autonomous_agent.reference_velocity
                new_status = (py_trees.common.Status.SUCCESS)
            ##########
        
            #print("    status =",new_status)
            return new_status

        def terminate(self, new_status: py_trees.common.Status) -> None:
            #Nothing to clean up in this example.
            self.logger.debug(
                "%s.terminate()[%s->%s]"
                % (self.__class__.__name__, self.status, new_status)
            )
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def Construct_Behavior_Tree(self):
        ##########
        # Creation of the nodes/objects. 
        # Each object is an instance of the respective class.        
        condition_destination_reached_1 = self.Condition_Destination_Reached(self, "condition_destination_reached_1")
        condition_destination_reached_1.setup()
        
        condition_destination_reached_2 = self.Condition_Destination_Reached(self, "condition_destination_reached_2")
        condition_destination_reached_2.setup()

        action_set_destination_equal_to_other_surveillance_position_s = self.Action_Set_Destination_Equal_To_Other_Surveillance_Position_S(self, "action_set_destination_equal_to_other_surveillance_position_s")
        action_set_destination_equal_to_other_surveillance_position_s.setup()
        
        action_navigate_to_destination_1_a = self.Action_Navigate_To_Destination_A(self, "action_navigate_to_destination_1_a")
        action_navigate_to_destination_1_a.setup()
        
        action_navigate_to_destination_2_a = self.Action_Navigate_To_Destination_A(self, "action_navigate_to_destination_2_a")
        action_navigate_to_destination_2_a.setup()
        
        condition_low_energy_level = self.Condition_Low_Energy_Level(self, "condition_low_energy_level")
        condition_low_energy_level.setup()
        
        action_set_destination_equal_to_refueling_position_s = self.Action_Set_Destination_Equal_To_Refueling_Position_S(self, "action_set_destination_equal_to_refueling_position_s")
        action_set_destination_equal_to_refueling_position_s.setup()
        
        action_set_destination_equal_to_current_surveillance_position_s = self.Action_Set_Destination_Equal_To_Current_Surveillance_Position_S(self, "action_set_destination_equal_to_current_surveillance_position_s")
        action_set_destination_equal_to_current_surveillance_position_s.setup()
        
        action_refuel_a = self.Action_Refuel_A(self, "action_refuel_a")
        action_refuel_a.setup()
        
        condition_radiation_detected = self.Condition_Radiation_Detected(self,"condition_radiation_detected")
        condition_radiation_detected.setup()
        
        action_avoid_radiation_zone_s = self.Action_Avoid_Radiation_Zone_S(self,"action_avoid_radiation_zone_s")
        action_avoid_radiation_zone_s.setup()
        
        condition_high_damage_level = self.Condition_High_Damage_Level(self,"condition_high_damage_level")
        condition_high_damage_level.setup()
        
        action_repair_damage_a = self.Action_Repair_Damage_A(self,"action_repair_damage_a")
        action_repair_damage_a.setup()
        
        #------------------------
        # Control and decorator nodes
        # O nó raiz já foi definido como sendo um reactive fallback (reactive selector).
        root = py_trees.composites.Selector(name="Selector", memory=False)
        
        reactive_sequence = py_trees.composites.Sequence(name="Sequence", memory=False)
        
        sequence1 = py_trees.composites.Sequence(name="Sequence", memory=True)
        
        sequence2 = py_trees.composites.Sequence(name="Sequence", memory=True)
        
        sequence3 = py_trees.composites.Sequence(name="Sequence", memory=True)
        
        sequence4 = py_trees.composites.Sequence(name="Sequence", memory=True)
        
        name="Inverter"
        inverter = py_trees.decorators.Inverter(name, condition_destination_reached_1)
        
        fallback1 = py_trees.composites.Selector(name="Selector", memory=True)
        
        reactive_fallback1 = py_trees.composites.Selector(name="Selector", memory=False)
        #------------------------        
        ##########
        
        ##########
        # Construction of the behavior tree through the created objects. 
        # Observation: the root node is the first node of the behavior tree.
                
        # INÍCIO DO LAB 2
        # O aluno deve criar a classe Action_Avoid_Radiation_Zone_S.
        # O aluno deve construir a árvore de comportamento utilizando os objetos definidos anteriormente.
        # O aluno pode usar o tutorial 1 para preparação do ambiente de trabalho.
        # O aluno pode usar os tutoriais 2 e 3 para construção de árvores de comportamento com a biblioteca py-trees.
        # O resultado gráfico da simulação é armazenado no arquivo "simulation TE276 LAB2.gif". 
        # Recomenda-se convertê-lo para um vídeo mp4 para facilitar a avaliação de resultados.
        # O aluno pode utilizar o seguinte site para conversão:
        # https://convertio.co/pt/gif-mp4/
        
        
        
        
        # FIM DO LAB 2
        #*******************************************************************************************************
        #*******************************************************************************************************
        #*******************************************************************************************************
        #*******************************************************************************************************                
        ##########
        
        self.behavior_tree_root = root
    #----------------------------------------------------------------------------------------------------------------------
    
    
    #----------------------------------------------------------------------------------------------------------------------
    def Decision_Making(self):
    
        self.behavior_tree_root.tick_once()
    #----------------------------------------------------------------------------------------------------------------------



        
#----------------------------------------------------------------------------------------------------------------------
# Auxiliary methods for the simulation.
#----------------------------------------------------------------------------------------------------------------------
def Store_Simulation_Data():
    global simulation_data
    global number_autonomous_agents
    global list_autonomous_agents
    
    for index_autonomous_agent in range(number_autonomous_agents):
        
        autonomous_agent = list_autonomous_agents[index_autonomous_agent]

        simulation_data[autonomous_agent.ID]['positions'].append([autonomous_agent.state['x'],autonomous_agent.state['y']])
        simulation_data[autonomous_agent.ID]['orientation_angles'].append(autonomous_agent.state['orientation_angle'])
        simulation_data[autonomous_agent.ID]['energy_levels'].append(autonomous_agent.state['energy_level'])
        simulation_data[autonomous_agent.ID]['velocities'].append(autonomous_agent.state['velocity'])
        simulation_data[autonomous_agent.ID]['number_ticks'].append(autonomous_agent.blackboard['number_ticks'])
        simulation_data[autonomous_agent.ID]['indexes_current_surveillance_positions'].append(autonomous_agent.blackboard['index_current_surveillance_position'])
        simulation_data[autonomous_agent.ID]['damage_levels'].append(autonomous_agent.state['damage_level'])

    
def Show_Current_State(autonomous_agent):
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>  t =", t)
    print("----------")
    print("CURRENT STATE")
    print("==> x =", autonomous_agent.state['x'])
    print("==> y =", autonomous_agent.state['y'])
    print("==> orientation angle =", autonomous_agent.state['orientation_angle'])
    print("==> velocity =", autonomous_agent.state['velocity'])
    print("==> energy level =", autonomous_agent.state['energy_level'])
    print("==> damage level =", autonomous_agent.state['damage_level'])
    print("==> number of ticks =", autonomous_agent.blackboard['number_ticks'])
    distance_agent_destination = math.sqrt(
        (autonomous_agent.state['x']-autonomous_agent.blackboard['x_current_destination_position'])*
        (autonomous_agent.state['x']-autonomous_agent.blackboard['x_current_destination_position']) +
        (autonomous_agent.state['y']-autonomous_agent.blackboard['y_current_destination_position'])*
        (autonomous_agent.state['y']-autonomous_agent.blackboard['y_current_destination_position']))
    print("==> distance between the agent and the destination =", distance_agent_destination)
    print("==> index of the current surveillance position =", autonomous_agent.blackboard['index_current_surveillance_position'])
    print("----------\n")

    
def Graphical_Part():
    global simulation_data
    global number_autonomous_agents
    global list_autonomous_agents
    
    print("----------------------------------------------------------------")
    print("GRAPHICAL PART")
    print("----------------------------------------------------------------")
    
    for index_autonomous_agent in range(number_autonomous_agents):   
        
        ID_autonomous_agent = list_autonomous_agents[index_autonomous_agent].ID
    
        simulation_data[ID_autonomous_agent]['positions'] = numpy.array(simulation_data[ID_autonomous_agent]['positions'])
        simulation_data[ID_autonomous_agent]['orientation_angles'] = numpy.array(simulation_data[ID_autonomous_agent]['orientation_angles'])
        simulation_data[ID_autonomous_agent]['energy_levels'] = numpy.array(simulation_data[ID_autonomous_agent]['energy_levels'])
        simulation_data[ID_autonomous_agent]['velocities'] = numpy.array(simulation_data[ID_autonomous_agent]['velocities'])
        simulation_data[ID_autonomous_agent]['number_ticks'] = numpy.array(simulation_data[ID_autonomous_agent]['number_ticks'])
        simulation_data[ID_autonomous_agent]['indexes_current_surveillance_positions'] = numpy.array(simulation_data[ID_autonomous_agent]['indexes_current_surveillance_positions'])
        simulation_data[ID_autonomous_agent]['damage_levels'] = numpy.array(simulation_data[ID_autonomous_agent]['damage_levels'])
        
        print("simulation_data["+ID_autonomous_agent+"]['positions'].shape =",simulation_data[ID_autonomous_agent]['positions'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['orientation_angles'].shape =",simulation_data[ID_autonomous_agent]['orientation_angles'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['energy_levels'].shape =",simulation_data[ID_autonomous_agent]['energy_levels'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['velocities'].shape =",simulation_data[ID_autonomous_agent]['velocities'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['number_ticks'].shape =", simulation_data[ID_autonomous_agent]['number_ticks'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['indexes_current_surveillance_positions'].shape =", simulation_data[ID_autonomous_agent]['indexes_current_surveillance_positions'].shape)
        print("simulation_data["+ID_autonomous_agent+"]['damage_levels'].shape =", simulation_data[ID_autonomous_agent]['damage_levels'].shape)

    print("\n|please wait| creating the graphical part...")
    
    refueling_position = numpy.array([[list_autonomous_agents[0].blackboard['x_refueling_position']],[list_autonomous_agents[0].blackboard['y_refueling_position']]])

    surveillance_position1 = numpy.array([[list_autonomous_agents[0].blackboard['x_surveillance_position_1']],[list_autonomous_agents[0].blackboard['y_surveillance_position_1']]])
    surveillance_position2 = numpy.array([[list_autonomous_agents[0].blackboard['x_surveillance_position_2']],[list_autonomous_agents[0].blackboard['y_surveillance_position_2']]])

    camera = Camera(pyplot.figure(figsize=(10, 10)))

    pyplot.xlim(0,100)
    pyplot.ylim(0,100)
    
    step = 10
    for i in range(0, number_iterations, step):
        
        pyplot.scatter(*refueling_position, c='cyan', s=100)  
        pyplot.scatter(*surveillance_position1, c='black', s=100, marker='x')
        pyplot.scatter(*surveillance_position2, c='black', s=100, marker='x')
        
        if (environment["radiation"]["activated_radiation_source"] == True):
            for index_radiation_source in range(len(environment["radiation"]["radiation sources"])):
                pyplot.scatter(environment["radiation"]["radiation sources"][index_radiation_source][1],
                               environment["radiation"]["radiation sources"][index_radiation_source][2],
                               c='red', s=1260*environment["radiation"]["radiation sources"][index_radiation_source][3])
                pyplot.scatter(environment["radiation"]["radiation sources"][index_radiation_source][1],
                               environment["radiation"]["radiation sources"][index_radiation_source][2],
                               c='white', s=1240*environment["radiation"]["radiation sources"][index_radiation_source][3])
                pyplot.scatter(environment["radiation"]["radiation sources"][index_radiation_source][1],
                               environment["radiation"]["radiation sources"][index_radiation_source][2],
                               c='red', marker='1', s=50)
        
        pyplot.text(50,111,"STATE", weight="bold",fontsize=10)
        
        pyplot.text(0,105,"BLACKBOARD", weight="bold",fontsize=10)
        
        # Show the ID of the first autonomous agent.
        pyplot.text(0,111,list_autonomous_agents[0].ID,weight="bold",fontsize=20)
        
        iteration = "ITERATION = " + str(i)
        pyplot.text(0,108,iteration, weight="bold",fontsize=10)
        
        # State visualization of the first autonomous agent.
        string_autonomous_agent_energy_level = "energy level = " + str(simulation_data[list_autonomous_agents[0].ID]['energy_levels'][i])
        pyplot.text(50,101,string_autonomous_agent_energy_level,fontsize=10)
        
        string_autonomous_agent_damage_level = "damage level = " + str(simulation_data[list_autonomous_agents[0].ID]['damage_levels'][i])
        pyplot.text(50,103,string_autonomous_agent_damage_level,fontsize=10)
    
        string_autonomous_agent_orientation_angle = "orientation angle = " + str(simulation_data[list_autonomous_agents[0].ID]['orientation_angles'][i])
        pyplot.text(50,105,string_autonomous_agent_orientation_angle,fontsize=10)
    
        string_autonomous_agent_velocity = "velocity = " + str(simulation_data[list_autonomous_agents[0].ID]['velocities'][i])
        pyplot.text(50,107,string_autonomous_agent_velocity,fontsize=10)
    
        string_autonomous_agent_position = "position = ( " + str(simulation_data[list_autonomous_agents[0].ID]['positions'][i][0]) + " , " + str(simulation_data[list_autonomous_agents[0].ID]['positions'][i][1]) + " )"
        pyplot.text(50,109,string_autonomous_agent_position,fontsize=10)
    
        # Blackboard visualization of the first autonomous agent.
        string_autonomous_agent_number_ticks = "number of ticks = " + str(simulation_data[list_autonomous_agents[0].ID]['number_ticks'][i])
        pyplot.text(0,101,string_autonomous_agent_number_ticks,fontsize=10)
    
        string_autonomous_agent_index_current_surveillance_position = "index of the current surveillance position = " + str(simulation_data[list_autonomous_agents[0].ID]['indexes_current_surveillance_positions'][i])
        pyplot.text(0,103,string_autonomous_agent_index_current_surveillance_position,fontsize=10)
        
        # Show each autonomous agent in the environment.
        for index_autonomous_agent in range(number_autonomous_agents):
            
            ID_autonomous_agent = list_autonomous_agents[index_autonomous_agent].ID
                  
            pyplot.scatter(simulation_data[ID_autonomous_agent]['positions'][i][0], simulation_data[ID_autonomous_agent]['positions'][i][1], c='blue', s=50)
    
            # Calculate the future auxiliary position to indicate the autonomous_agent orientation.
            x_autonomous_agent_future_position = simulation_data[ID_autonomous_agent]['positions'][i][0] + 0.9*math.cos(simulation_data[ID_autonomous_agent]['orientation_angles'][i]*math.pi/180.0)
            y_autonomous_agent_future_position = simulation_data[ID_autonomous_agent]['positions'][i][1] + 0.9*math.sin(simulation_data[ID_autonomous_agent]['orientation_angles'][i]*math.pi/180.0)
            pyplot.scatter(x_autonomous_agent_future_position, y_autonomous_agent_future_position, c='red', s=10)
            
            pyplot.text(simulation_data[ID_autonomous_agent]['positions'][i][0],simulation_data[ID_autonomous_agent]['positions'][i][1]+1,str(index_autonomous_agent+1),fontsize=10)
    
        camera.snap()

    anim = camera.animate(blit=True)
    init_time = time.time()
    anim.save('simulation TE276 LAB2.gif',writer='pillow')
    end_time = time.time()
    print("time interval need to save =",(end_time - init_time)," seconds")
#----------------------------------------------------------------------------------------------------------------------

        
        
        
        
        
        
        
#----------------------------------------------------------------------------------------------------------------------
# SIMULATION
#----------------------------------------------------------------------------------------------------------------------

# Simulation init time.
simulation_init_time = time.time()

# List of the autonomous agents.
list_autonomous_agents = []

# Creation of the drones. Here, we can add the objects of our autonomous agent classes :)
list_autonomous_agents.append(Drone("drone1"))

# Specification of the initial position of each drone.
# drone1
list_autonomous_agents[0].state['x'] = 10.0
list_autonomous_agents[0].state['y'] = 50.0


# Construction of the behavior tree for each drone.
for index_autonomous_agent in range(len(list_autonomous_agents)):
    list_autonomous_agents[index_autonomous_agent].Construct_Behavior_Tree()

# Simulation data.
simulation_data = {}

# Creating the data store of the each drone.
for index_autonomous_agent in range(len(list_autonomous_agents)):
    simulation_data[list_autonomous_agents[index_autonomous_agent].ID] = {
        "positions":[],
        "orientation_angles":[],
        "energy_levels":[],
        "velocities":[],
        "number_ticks":[],
        "indexes_current_surveillance_positions":[],
        "damage_levels":[]        
    }

number_autonomous_agents = len(list_autonomous_agents)

number_iterations = 4000

t = 1
while(t <= number_iterations):
    
    for index_autonomous_agent in range(number_autonomous_agents):
        
        # Autonomous agent of the position index_autonomous_agent inside list_autonomous_agents.
        autonomous_agent = list_autonomous_agents[index_autonomous_agent]
        
        # Update the autonomous agent's state.
        autonomous_agent.Update_State(t)
        
        # Show the current state of the autonomous agent.
        Show_Current_State(autonomous_agent)
        
        # Decision making
        autonomous_agent.Decision_Making()
        
        # Required velocity and required orientation angle calculated by the decision making process are implemented by the autonomous agent's control system.
        autonomous_agent.Control_System()
    
    # Store the simulation data.
    Store_Simulation_Data()
    
    t = t + 1
    #print("\n")

# Simulation end time.
simulation_end_time = time.time()
print("--> Simulation Finished <--  simulation time =",(simulation_end_time - simulation_init_time)," seconds")
    
Graphical_Part()
